In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [28]:
df_wellcome = pd.read_csv('../data/processed/wellcomeMap.csv')
df_wellcome.head()

,name,formatted_address,place_id,rating,user_ratings_total,types,latitude,longitude
0,wellcome,"125, Metro Harbour View Plaza, 8 Fuk Lee St, T...",ChIJfzdi4rAABDQRYsiJViBAKIs,3.7,259,"['supermarket', 'grocery_or_supermarket', 'foo...",22.324410,114.160454
1,Wellcome,"84-90號 Queen's Road Central, Central, Hong Kong",ChIJ5b0oWHsABDQRJUwc57avQ94,3.8,362,"['supermarket', 'grocery_or_supermarket', 'foo...",22.283245,114.155105
2,Wellcome,"319 Nathan Rd, Jordan, Hong Kong",ChIJG3W4guwABDQRsYiKQEyeUvU,3.6,275,"['supermarket', 'grocery_or_supermarket', 'foo...",22.306436,114.171362
3,Wellcome,"Shop No.23, G/F, Hoi Fu Shopping Centre, 2 Hoi...",ChIJvdn9MLkABDQRbEofjcY9gsc,3.6,215,"['supermarket', 'grocery_or_supermarket', 'foo...",22.315173,114.164415
4,Wellcome,"G/F, TG Place, 8 Shing Yip St, Kwun Tong, Hong...",ChIJabudQ0UBBDQRF2ggpcEzLiE,3.5,182,"['supermarket', 'grocery_or_supermarket', 'foo...",22.310417,114.225745


## BusStop

In [29]:
df_busStop = pd.read_csv('../data/processed/busStop.csv')
df_busStop.head()

,id,STOP_ID,X,Y,LAST_UPDATE_DATE_STR,geometry,longitude,latitude
0,GS_BUS_STOP_VIEW.134,134,836298,815477,10/24/2022,POINT (114.17716544074156 22.278157267915223),114.177165,22.278157
1,GS_BUS_STOP_VIEW.135,135,836643,815593,10/24/2022,POINT (114.18051304472341 22.279204780631602),114.180513,22.279205
2,GS_BUS_STOP_VIEW.136,136,837048,815708,10/24/2022,POINT (114.1844428939664 22.280243167292028),114.184443,22.280243
3,GS_BUS_STOP_VIEW.137,137,837583,815779,10/24/2022,POINT (114.1896341986785 22.280884016074378),114.189634,22.280884
4,GS_BUS_STOP_VIEW.139,139,837920,816521,10/24/2022,POINT (114.19290497970104 22.287584455024394),114.192905,22.287584


In [30]:
from haversine import haversine, Unit

# Define a function to count the number of bus stops within a 1km radius of each shop
def count_bus_stops_within_1km(row, radius = 1000):
    shop_loc = (row['latitude'], row['longitude'])
    num_stops = 0
    for index, stop in df_busStop.iterrows():
        stop_loc = (stop['latitude'], stop['longitude'])
        distance = haversine(shop_loc, stop_loc, unit=Unit.METERS)
        if distance <= radius:  # 1km = 1000 meters
            num_stops += 1
    return num_stops

# Apply the function to each row in the shops dataframe and store the results in a new column
df_wellcome['num_bus_stops_within_1km'] = df_wellcome.apply(count_bus_stops_within_1km, axis=1)


In [31]:
df_wellcome.head()

,name,formatted_address,place_id,rating,user_ratings_total,types,latitude,longitude,num_bus_stops_within_1km
0,wellcome,"125, Metro Harbour View Plaza, 8 Fuk Lee St, T...",ChIJfzdi4rAABDQRYsiJViBAKIs,3.7,259,"['supermarket', 'grocery_or_supermarket', 'foo...",22.324410,114.160454,117
1,Wellcome,"84-90號 Queen's Road Central, Central, Hong Kong",ChIJ5b0oWHsABDQRJUwc57avQ94,3.8,362,"['supermarket', 'grocery_or_supermarket', 'foo...",22.283245,114.155105,116
2,Wellcome,"319 Nathan Rd, Jordan, Hong Kong",ChIJG3W4guwABDQRsYiKQEyeUvU,3.6,275,"['supermarket', 'grocery_or_supermarket', 'foo...",22.306436,114.171362,98
3,Wellcome,"Shop No.23, G/F, Hoi Fu Shopping Centre, 2 Hoi...",ChIJvdn9MLkABDQRbEofjcY9gsc,3.6,215,"['supermarket', 'grocery_or_supermarket', 'foo...",22.315173,114.164415,94
4,Wellcome,"G/F, TG Place, 8 Shing Yip St, Kwun Tong, Hong...",ChIJabudQ0UBBDQRF2ggpcEzLiE,3.5,182,"['supermarket', 'grocery_or_supermarket', 'foo...",22.310417,114.225745,78


# Mtr Station

In [32]:
df_mtr = pd.read_csv('../data/processed/mtr.csv')
df_mtr

,name,description,longitude,latitude,altitude
0,Asia World-Expo,"<img src=""https://lh3.googleusercontent.com/um...",113.942304,22.321828,0.0
1,Airport (HKG),"<img src=""https://lh3.googleusercontent.com/um...",113.936548,22.315947,0.0
2,Tsing Yi,"<img src=""https://lh3.googleusercontent.com/um...",114.107459,22.358321,0.0
3,Kowloon,"<img src=""https://lh3.googleusercontent.com/um...",114.161500,22.304701,0.0
4,Hong Kong,"<img src=""https://lh3.googleusercontent.com/um...",114.158179,22.284872,0.0
...,...,...,...,...,...
93,Tai Shui Hang,"<img src=""https://lh3.googleusercontent.com/um...",114.222622,22.408241,0.0
94,Shek Mun,"<img src=""https://lh3.googleusercontent.com/um...",114.208653,22.387887,0.0
95,City One,"<img src=""https://lh3.googleusercontent.com/um...",114.203535,22.382956,0.0
96,Sha Tin Wai,"<img src=""https://lh3.googleusercontent.com/um...",114.194770,22.376944,0.0


In [33]:
from haversine import haversine, Unit

# Define a function to count the number of bus stops within a 1km radius of each shop
def count_mtr_station_within_1km(row, radius = 1000):
    shop_loc = (row['latitude'], row['longitude'])
    num_mtr = 0
    for index, stop in df_mtr.iterrows():
        mtr_loc = (stop['latitude'], stop['longitude'])
        distance = haversine(shop_loc, mtr_loc, unit=Unit.METERS)
        if distance <= radius:  # 1km = 1000 meters
            num_mtr += 1
    return num_mtr

# Apply the function to each row in the shops dataframe and store the results in a new column
df_wellcome['num_mtr_station_within_1km'] = df_wellcome.apply(count_mtr_station_within_1km, axis=1)


In [34]:
df_wellcome

,name,formatted_address,place_id,rating,user_ratings_total,types,latitude,longitude,num_bus_stops_within_1km,num_mtr_station_within_1km
0,wellcome,"125, Metro Harbour View Plaza, 8 Fuk Lee St, T...",ChIJfzdi4rAABDQRYsiJViBAKIs,3.7,259,"['supermarket', 'grocery_or_supermarket', 'foo...",22.324410,114.160454,117,6
1,Wellcome,"84-90號 Queen's Road Central, Central, Hong Kong",ChIJ5b0oWHsABDQRJUwc57avQ94,3.8,362,"['supermarket', 'grocery_or_supermarket', 'foo...",22.283245,114.155105,116,5
2,Wellcome,"319 Nathan Rd, Jordan, Hong Kong",ChIJG3W4guwABDQRsYiKQEyeUvU,3.6,275,"['supermarket', 'grocery_or_supermarket', 'foo...",22.306436,114.171362,98,3
3,Wellcome,"Shop No.23, G/F, Hoi Fu Shopping Centre, 2 Hoi...",ChIJvdn9MLkABDQRbEofjcY9gsc,3.6,215,"['supermarket', 'grocery_or_supermarket', 'foo...",22.315173,114.164415,94,5
4,Wellcome,"G/F, TG Place, 8 Shing Yip St, Kwun Tong, Hong...",ChIJabudQ0UBBDQRF2ggpcEzLiE,3.5,182,"['supermarket', 'grocery_or_supermarket', 'foo...",22.310417,114.225745,78,3
...,...,...,...,...,...,...,...,...,...,...
57,Wellcome,"Wearbest Building, 66 To Kwa Wan Rd, To Kwa Wa...",ChIJcbEJNtgABDQRoRteMKfSZTI,3.7,318,"['supermarket', 'grocery_or_supermarket', 'foo...",22.315762,114.188902,90,0
58,惠康 Wellcome,"Portion of Flats A-D, G/F, Yuen Shing Bldg, 9,...",ChIJbSauYecABDQR7BkK69nBuDg,3.7,94,"['supermarket', 'grocery_or_supermarket', 'foo...",22.306573,114.183912,81,1
59,Wellcome Supermarket - Princeton Tower,"80 Des Voeux Rd W, Sheung Wan, Hong Kong",ChIJvQA8CH8ABDQRwY85G-qMfhU,3.6,85,"['supermarket', 'grocery_or_supermarket', 'foo...",22.287621,114.145976,92,1
60,Wellcome - Yat Tung Superstore,"Shop No 110, 1/F Eaton Shopping Centre, Yat Tu...",ChIJibJ9J6HiAzQR6f60A8w4neE,3.6,214,"['supermarket', 'grocery_or_supermarket', 'foo...",22.281394,113.934796,28,0


# District

In [35]:
df_district = pd.read_csv('../data/processed/districtAll.csv')
df_district

,district,latitude,longitude,Total,Average household sie,Median income
0,Central and Western District,22.28219,114.14486,87700.0,2.6,42000.0
1,Eastern,22.27722,114.22519,188400.0,2.8,31500.0
2,Islands District,22.26382,113.96038,67400.0,2.7,28800.0
3,Kowloon City District,22.32866,114.19121,146500.0,2.7,29700.0
4,Kwai Tsing,22.35288,114.10004,176600.0,2.7,23300.0
5,Kwon Tong,22.31326,114.22581,247400.0,2.7,22200.0
6,North,22.49471,114.13812,111300.0,2.7,23300.0
7,Sai Kung Area,22.38198,114.27017,166900.0,2.9,37200.0
8,Sha Tin,22.38715,114.19534,245800.0,2.8,27100.0
9,Sham Shui Po,22.32989,114.16250,164700.0,2.6,22000.0


In [36]:
import pandas as pd
from haversine import haversine, Unit


# Define a function to calculate the distance between two points using the Haversine formula
def calc_distance(lat1, lon1, lat2, lon2):
    return haversine((lat1, lon1), (lat2, lon2), unit=Unit.KILOMETERS)

# Create a new column in the shops dataframe to store the district name
df_wellcome['district'] = np.nan

# Iterate through each shop and find the nearest district
for i, shop in df_wellcome.iterrows():
    min_distance = float('inf')
    district_name = ''
    for j, district in df_district.iterrows():
        distance = calc_distance(shop['latitude'], shop['longitude'], district['latitude'], district['longitude'])
        if distance < min_distance:
            min_distance = distance
            district_name = district['district']
    df_wellcome.at[i, 'district'] = district_name



In [37]:
df_wellcome

,name,formatted_address,place_id,rating,user_ratings_total,types,latitude,longitude,num_bus_stops_within_1km,num_mtr_station_within_1km,district
0,wellcome,"125, Metro Harbour View Plaza, 8 Fuk Lee St, T...",ChIJfzdi4rAABDQRYsiJViBAKIs,3.7,259,"['supermarket', 'grocery_or_supermarket', 'foo...",22.324410,114.160454,117,6,Sham Shui Po
1,Wellcome,"84-90號 Queen's Road Central, Central, Hong Kong",ChIJ5b0oWHsABDQRJUwc57avQ94,3.8,362,"['supermarket', 'grocery_or_supermarket', 'foo...",22.283245,114.155105,116,5,Central and Western District
2,Wellcome,"319 Nathan Rd, Jordan, Hong Kong",ChIJG3W4guwABDQRsYiKQEyeUvU,3.6,275,"['supermarket', 'grocery_or_supermarket', 'foo...",22.306436,114.171362,98,3,Yau Tsim Mong
3,Wellcome,"Shop No.23, G/F, Hoi Fu Shopping Centre, 2 Hoi...",ChIJvdn9MLkABDQRbEofjcY9gsc,3.6,215,"['supermarket', 'grocery_or_supermarket', 'foo...",22.315173,114.164415,94,5,Yau Tsim Mong
4,Wellcome,"G/F, TG Place, 8 Shing Yip St, Kwun Tong, Hong...",ChIJabudQ0UBBDQRF2ggpcEzLiE,3.5,182,"['supermarket', 'grocery_or_supermarket', 'foo...",22.310417,114.225745,78,3,Kwon Tong
...,...,...,...,...,...,...,...,...,...,...,...
57,Wellcome,"Wearbest Building, 66 To Kwa Wan Rd, To Kwa Wa...",ChIJcbEJNtgABDQRoRteMKfSZTI,3.7,318,"['supermarket', 'grocery_or_supermarket', 'foo...",22.315762,114.188902,90,0,Kowloon City District
58,惠康 Wellcome,"Portion of Flats A-D, G/F, Yuen Shing Bldg, 9,...",ChIJbSauYecABDQR7BkK69nBuDg,3.7,94,"['supermarket', 'grocery_or_supermarket', 'foo...",22.306573,114.183912,81,1,Yau Tsim Mong
59,Wellcome Supermarket - Princeton Tower,"80 Des Voeux Rd W, Sheung Wan, Hong Kong",ChIJvQA8CH8ABDQRwY85G-qMfhU,3.6,85,"['supermarket', 'grocery_or_supermarket', 'foo...",22.287621,114.145976,92,1,Central and Western District
60,Wellcome - Yat Tung Superstore,"Shop No 110, 1/F Eaton Shopping Centre, Yat Tu...",ChIJibJ9J6HiAzQR6f60A8w4neE,3.6,214,"['supermarket', 'grocery_or_supermarket', 'foo...",22.281394,113.934796,28,0,Islands District


In [38]:


# create a map centered at Hong Kong
m = folium.Map(location=[22.3193, 114.1694], zoom_start=11)

# iterate through the rows of the dataframe and add markers to the map
for index, row in df_wellcome.iterrows():
    name = row['formatted_address']
    lat = row['latitude']
    lon = row['longitude']
    popup = folium.Popup(name)
    marker = folium.Marker(location=[lat, lon], popup=popup)
    marker.add_to(m)

# display the map
m


In [39]:
df_wellcome.to_csv("../data/processed/wellcomeAnalysis.csv",index=False)